In [1]:
import sys
sys.path.append('C:\\projects\\UTNCE\\src\\utnce')

In [2]:
from simplify import *
from prepare import *
from routing import *
from percolation_plot import *
from generate import *
from transfer import *
from copy import deepcopy
from shapely.geometry import Point
from pyproj import Proj, Transformer

In [3]:
osm_path = "C:\\projects\\UTNCE\\data\\Amsterdam.large.pbf"

# Subway-Amsterdam

## Generate subway networks

In [4]:
# Secondly, creates a dictionary that maps start station names to their corresponding destinations
def start_station_dict(routes_file):
    """
    Creates a dictionary mapping start station names to their corresponding destinations.

    Args:
        routes_file (pandas.DataFrame): A DataFrame containing route information.

    Returns:
        dict: A dictionary mapping start station names to their destinations.
    """
    # Initialize an empty dictionary to store the mappings
    start_station_name_dict = {} 

    # Iterate over each row in the DataFrame
    for index, row in routes_file.iterrows():  
        # Get the start station name from the 'name' column
        key = row['name'] 
        # Get the destination from the 'to' column
        value = row['to']  
        # Add the mapping to the dictionary
        start_station_name_dict[key] = value  
    # Return the dictionary    
    return start_station_name_dict

    
# Creates a dictionary mapping line names to their corresponding indices
def line_dict(routes_file):
    """
    Creates a dictionary mapping line names to their corresponding indices.

    Args:
        routes_file (pandas.DataFrame): A DataFrame containing route information.

    Returns:
        dict: A dictionary mapping line names to their corresponding indices.
    """
    # Initialize an empty dictionary to store line names and indices
    line_num_dict = {}  

    # Iterate over each row in the routes_file DataFrame
    for index, row in routes_file.iterrows():
        # Get the value in the 'name' column of the current row
        key = row['name']  
        # Add an entry to the dictionary with the line name as the key and the index as the value
        line_num_dict[key] = index  
    
    # Return the line_num_dict dictionary as the output
    return line_num_dict  


# Create a list for all_station with selected columns
def all_station_list(all_stations_file):
    """
    Extracts relevant information from a GeoDataFrame containing station data.

    Args:
        all_stations_file (GeoDataFrame): A GeoDataFrame containing station information.

    Returns:
        GeoDataFrame: A GeoDataFrame with selected columns ('name', 'geometry', 'geo_x', 'geo_y').

    """
    # Create a copy of the all_stations_file as a GeoDataFrame
    all_stations_file = gpd.GeoDataFrame(all_stations_file.copy())
    
    # Select specific columns ('name', 'geometry', 'geo_x', 'geo_y') and reset the index
    all_stations_name = all_stations_file[['name', 'geometry', 'geo_x', 'geo_y']].reset_index(drop=True)
    
    # Return the resulting GeoDataFrame with selected columns
    return all_stations_name

# # Sorting the stations on each route
# def order_route(first_stop, unordered_route):
#     new_order = []
#     remaining_route = unordered_route.drop(unordered_route[unordered_route.name == first_stop.name.values[0]].index).reset_index(drop=True)
#     tree = shapely.STRtree(remaining_route.geometry)
#     new_order.append(first_stop)
#     for iter_ in range(len(remaining_route)):
#         try:
#             if iter_ == 0:
#                 nearest_station = pd.DataFrame(remaining_route.iloc[tree.nearest(first_stop.geometry)[0]]).T
#                 new_order.append(nearest_station)
#                 remaining_route = remaining_route.drop(remaining_route[remaining_route.name == nearest_station.name.values[0]].index).reset_index(drop=True)
#                 tree = shapely.STRtree(remaining_route.geometry)
#             elif iter_ == 1:
#                 second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(nearest_station.geometry)[0]]).T
#                 new_order.append(second_station)
#                 remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
#                 tree = shapely.STRtree(remaining_route.geometry)
#             else:
#                 second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(second_station.geometry)[0]]).T
#                 new_order.append(second_station)
#                 remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
#                 tree = shapely.STRtree(remaining_route.geometry)
#         except TypeError:
#             pass  # pass 'NoneType' object is not subscriptable Error
#     return pd.concat(new_order).reset_index(drop=True)
    
# Sorting the stations on each route
def order_route(first_stop, unordered_route):
    """
    Orders the route based on the nearest stations.

    Args:
        first_stop (pandas.Series): The first stop to start the route(Destination actually).
        unordered_route (pandas.DataFrame): The unordered route containing stops.

    Returns:
        pandas.DataFrame: The ordered route based on the nearest stations.
    """
    new_order = []

    # Remove the first stop from the unordered route
    remaining_route = unordered_route.drop(unordered_route[unordered_route.name == first_stop.name.values[0]].index).reset_index(drop=True)

    # Build an STRtree index for the remaining route
    tree = shapely.strtree.STRtree(remaining_route.geometry)

    # Append the first stop to the new order
    new_order.append(first_stop)

    for iter_ in range(len(remaining_route)):
        try:
            if iter_ == 0:
                # Find the nearest station to the first stop
                nearest_station = pd.DataFrame(remaining_route.iloc[tree.nearest(first_stop.geometry)[0]]).T
                new_order.append(nearest_station)
                remaining_route = remaining_route.drop(remaining_route[remaining_route.name == nearest_station.name.values[0]].index).reset_index(drop=True)
                tree = shapely.strtree.STRtree(remaining_route.geometry)
            elif iter_ == 1:
                # Find the second nearest station to the first stop
                second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(nearest_station.geometry)[0]]).T
                new_order.append(second_station)
                remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
                tree = shapely.strtree.STRtree(remaining_route.geometry)
            else:
                # Find the next nearest station based on the previous station
                second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(second_station.geometry)[0]]).T
                new_order.append(second_station)
                remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
                tree = shapely.strtree.STRtree(remaining_route.geometry)
        except TypeError:
            pass  # Ignore 'NoneType' object is not subscriptable error

    # Concatenate the ordered stops into a DataFrame
    ordered_route = pd.concat(new_order).reset_index(drop=True)
    # Reverse the sequence to 
    ordered_route = ordered_route.iloc[::-1].reset_index(drop=True)
    return ordered_route


# Orders stations based on the tram line routes and start station names
def order_stations_inline(tram_line_dict, all_tram_stations_name, tram_routes, tram_start_station_name_dict):
    """
    Orders the tram stations based on the tram line routes and start station names.

    Args:
        tram_line_dict (dict): A dictionary containing tram line information.
        all_tram_stations_name (DataFrame): A DataFrame containing all tram station names.
        tram_routes (DataFrame): A DataFrame containing tram route information.
        tram_start_station_name_dict (dict): A dictionary mapping tram line names to their respective start station names.

    Returns:
        dict: A dictionary containing ordered tram station routes for each tram line.
    """
    
    # Create a copy of the tram_line_dict dictionary
    tram_stations_dict = tram_line_dict.copy()

    # Iterate over each key-value pair in tram_stations_dict
    for key, value in tram_stations_dict.items():
        value1 = value
        # Filter the all_tram_stations_name DataFrame to include only stations within the buffer of the corresponding tram route
        value2 = all_tram_stations_name.loc[all_tram_stations_name.within(tram_routes.iloc[value1].geometry.buffer(0.000001))]
        # Assign new IDs to the filtered stations
        value2['id'] = value2.reset_index().index
        # Update the value in tram_stations_dict to include only necessary columns and reset the index
        tram_stations_dict[key] = value2[['id', 'name', 'geometry', 'geo_x', 'geo_y']].reset_index(drop=True)

    # Create copies of tram_line_dict for storing tram stations in order and ordered routes
    tram_stations_inorder_dict = tram_line_dict.copy()
    tram_order_route_dict = tram_line_dict.copy()

    # Iterate over each line in tram_stations_inorder_dict
    for line in tram_stations_inorder_dict.keys():
        tram_stations_inorder = tram_stations_dict[line]
        # Find the start station for the current line and update tram_stations_inorder_dict with only that station
        for i in range(len(tram_stations_inorder)):
            if tram_stations_inorder.iloc[i]['name'] == tram_start_station_name_dict[line]:
                tram_stations_inorder_dict[line] = pd.DataFrame(tram_stations_inorder.iloc[i]).T
        # Call the order_route function to order the remaining stations and store the result in tram_order_route_dict
        tram_order_route_dict[line] = order_route(tram_stations_inorder_dict[line], tram_stations_dict[line])

    return tram_order_route_dict

In [5]:
# Subway/Metro
def prepare_metro(osm_path):
    """
    Prepares the metro network data based on the given OSM path.

    Args:
        osm_path (str): The path to the OpenStreetMap (OSM) file.

    Returns:
        tuple: A tuple containing the following metro network data:
            - city_sub_stations: Sub-stations in the city
            - edges: Edges (connections) between stations in the network
            - nodes: Nodes (stations) in the network
            - city_sub_routes: Sub-routes in the city
            - check_name: Result of the check operation
    """

    # Obtain sub-stations in the city
    city_sub_stations = sub_stations(osm_path)

    # Obtain subway network data
    city_sub_network = subway_network(osm_path)

    # Prepare the network edges and nodes based on sub-stations and subway network
    edges, nodes = prepare_network(city_sub_network, city_sub_stations)

    # Expand the edges if required
    edges = expand_edges(edges)

    # Retrieve sub-stations again (optional, as it was already obtained before)
    city_sub_stations = sub_stations(osm_path)

    # Obtain sub-routes in the city
    city_sub_routes = sub_routes(osm_path)

    # Sort the sub-routes
    city_sub_routes = sorted_routes(city_sub_routes)

    # Check the name and assign a column
    check_name = check_to_column(city_sub_routes, city_sub_stations)

    # Return all the obtained data
    return city_sub_stations, edges, nodes, city_sub_routes, check_name

def recheck_metro_routes(replacement_dict, city_sub_stations, city_sub_routes):
    """
    This function rechecks the metro routes based on the given inputs.

    Args:
    - city_sub_stations: Dataframe or data structure containing subway station information for the city.
    - replacement_dict: A dictionary used to replace certain values in the 'to' column of metro routes.
    - city_sub_routes: Dataframe or data structure containing metro route information for the city.

    Returns:
    - check_name: A list of boolean values indicating whether the 'to' column in the metro routes matches the station names.
    - city_sub_routes: a dataframe of routes after changing the start staion name according to city_sub_stations.
    """

    # Replace values in the 'to' column of the metro routes using the replacement dictionary
    city_sub_routes['to'] = city_sub_routes['to'].replace(replacement_dict, regex=True).str.strip()

    # Reset the index of the metro routes
    city_sub_routes = city_sub_routes.reset_index(drop=True)

    # Check if the values in the 'to' column of the metro routes match the station names
    check_name = check_to_column(city_sub_routes, city_sub_stations)

    return city_sub_routes, check_name

def metro(city_sub_stations, edges, nodes, city_sub_routes, city='Rotterdam'):
    """
    Performs various operations on the metro network data to calculate shortest paths and plot routes.

    Args:
        city_sub_stations (list): List of sub-stations in the city.
        edges (list): List of edges (connections) between stations in the network.
        nodes (list): List of nodes (stations) in the network.
        city_sub_routes (list): List of sub-routes in the city.
        city (str, optional): Name of the city. Defaults to 'Rotterdam'.

    Returns:
        tuple: A tuple containing the following calculated data:
            - city_sub_shortest_path_pairs: Shortest path pairs for each subway line.
            - city_sub_shortest_path_edges: Shortest path edges for each subway line.
            - city_sub_edges: Subway network edges for each subway line.
    """

    # Create a dictionary mapping start stations to their corresponding routes
    city_sub_start_station_name_dict = start_station_dict(city_sub_routes)

    # Create a dictionary mapping subway lines to their corresponding routes
    city_sub_line_dict = line_dict(city_sub_routes)

    # Create a list of all station names in the city's subway network
    city_all_sub_stations_name = all_station_list(city_sub_stations)

    # Create a dictionary mapping subway lines to the ordered list of stations on each line
    city_sub_order_route_dict = order_stations_inline(city_sub_line_dict, city_all_sub_stations_name, city_sub_routes, city_sub_start_station_name_dict)

    # Create a dictionary mapping subway lines to the pairs of station IDs on each line
    city_sub_order_id_pairs = id_pairs_inline(city_sub_line_dict, city_sub_order_route_dict, nodes)

    # Create a graph representing the subway network using the extracted edges and nodes
    G = create_ground_graph(edges, nodes)

    # Calculate the shortest path pairs for each subway line
    city_sub_shortest_path_pairs = city_sub_order_id_pairs.copy()
    duplicate_row_count = city_sub_order_id_pairs.copy()
    city_sub_shortest_path_edges = city_sub_order_id_pairs.copy()
    city_sub_edges = city_sub_order_id_pairs.copy()

    for line in city_sub_order_id_pairs.keys():
        # Calculate all shortest paths for the current line using the subway network edges
        city_sub_shortest_path_pairs[line] = all_shortest_paths(G, city_sub_order_id_pairs[line], edges)

        # Count the duplicate rows, calculate the weighted edges for shortest paths,
        # and update the subway network edges for the current line
        duplicate_row_count[line], city_sub_shortest_path_edges[line], city_sub_edges[line] = edges_with_count_weight(city_sub_shortest_path_pairs[line], edges)
        sub_edges = city_sub_edges[line]
    
    # # Plot the subway routes with even indices
    # plot_routes_even(city_sub_routes, edges, city_sub_shortest_path_edges)

    # # Plot the subway routes with odd indices
    # plot_routes_odd(city_sub_routes, edges, city_sub_shortest_path_edges)
        
    # Plot all subway routes
    plot_routes(city_sub_routes, edges, city_sub_shortest_path_edges)

    # Return the calculated data
    return city_sub_order_route_dict, city_sub_shortest_path_pairs, city_sub_shortest_path_edges, sub_edges #city_sub_edges

In [6]:
sub_stations, edges, nodes, sub_routes, check_name = prepare_metro(osm_path)

topology: 100%|████████████████████████████████████████████████████████████████████| 962/962 [00:00<00:00, 7771.46it/s]


In [7]:
replacement_dict = {
     'Amsterdam Centraal': 'Centraal Station',
}


sub_routes, check_name = recheck_metro_routes(replacement_dict, sub_stations, sub_routes)

In [8]:
sub_routes

,osm_id,route,to,name,ref,network,service,geometry
0,18683,subway,Isolatorweg,Metro 50: Gein => Isolatorweg,50,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.98996 52.29640, 4.98959 52..."
1,4515354,subway,Gein,Metro 50: Isolatorweg => Gein,50,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.83880 52.38842, 4.83897 52..."
2,31249,subway,Isolatorweg,Metro 51: Centraal Station => Isolatorweg,51,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.90193 52.37689, 4.90202 52..."
3,4515355,subway,Centraal Station,Metro 51: Isolatorweg => Centraal Station,51,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.83880 52.38842, 4.83897 52..."
4,8466667,subway,Zuid,Metro 52: Noord => Zuid,52,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.93265 52.40228, 4.93269 52..."
5,8466671,subway,Noord,Metro 52: Zuid => Noord,52,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.87423 52.33922, 4.87552 52..."
6,18532,subway,Gaasperplas,Metro 53: Centraal Station => Gaasperplas,53,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.90193 52.37689, 4.90202 52..."
7,4515356,subway,Centraal Station,Metro 53: Gaasperplas => Centraal Station,53,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.98373 52.31201, 4.98364 52..."
8,18423,subway,Gein,Metro 54: Centraal Station => Gein,54,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.90193 52.37689, 4.90202 52..."
9,4515357,subway,Centraal Station,Metro 54: Gein => Centraal Station,54,Stadsvervoer Amsterdam,None,"MULTILINESTRING ((4.98996 52.29640, 4.98959 52..."


In [9]:
city_sub_start_station_name_dict = start_station_dict(sub_routes)
city_sub_start_station_name_dict

{'Metro 50: Gein => Isolatorweg': 'Isolatorweg',
 'Metro 50: Isolatorweg => Gein': 'Gein',
 'Metro 51: Centraal Station => Isolatorweg': 'Isolatorweg',
 'Metro 51: Isolatorweg => Centraal Station': 'Centraal Station',
 'Metro 52: Noord => Zuid': 'Zuid',
 'Metro 52: Zuid => Noord': 'Noord',
 'Metro 53: Centraal Station => Gaasperplas': 'Gaasperplas',
 'Metro 53: Gaasperplas => Centraal Station': 'Centraal Station',
 'Metro 54: Centraal Station => Gein': 'Gein',
 'Metro 54: Gein => Centraal Station': 'Centraal Station'}

In [10]:
city_sub_line_dict = line_dict(sub_routes)
city_sub_line_dict

{'Metro 50: Gein => Isolatorweg': 0,
 'Metro 50: Isolatorweg => Gein': 1,
 'Metro 51: Centraal Station => Isolatorweg': 2,
 'Metro 51: Isolatorweg => Centraal Station': 3,
 'Metro 52: Noord => Zuid': 4,
 'Metro 52: Zuid => Noord': 5,
 'Metro 53: Centraal Station => Gaasperplas': 6,
 'Metro 53: Gaasperplas => Centraal Station': 7,
 'Metro 54: Centraal Station => Gein': 8,
 'Metro 54: Gein => Centraal Station': 9}

In [11]:
city_all_sub_stations_name = all_station_list(sub_stations)
city_all_sub_stations_name

,name,geometry,geo_x,geo_y
0,Wibautstraat,POINT (4.91205 52.35459),4.912049,52.354595
1,Zuid,POINT (4.87398 52.33904),4.873978,52.339035
2,Station Sloterdijk,POINT (4.83898 52.38904),4.838979,52.389038
3,Isolatorweg,POINT (4.85060 52.39510),4.850596,52.395103
4,De Vlugtlaan,POINT (4.83806 52.37946),4.838056,52.379461
...,...,...,...,...
159,Waterlooplein,POINT (4.90361 52.36664),4.903606,52.366636
160,Waterlooplein,POINT (4.90375 52.36669),4.903753,52.366690
161,Nieuwmarkt,POINT (4.90094 52.37142),4.900938,52.371424
162,Weesperplein,POINT (4.90792 52.36133),4.907918,52.361326


In [12]:
def order_route(first_stop, unordered_route):
    """
    Orders the route based on the nearest stations.

    Args:
        first_stop (pandas.Series): The first stop to start the route(Destination actually).
        unordered_route (pandas.DataFrame): The unordered route containing stops.

    Returns:
        pandas.DataFrame: The ordered route based on the nearest stations.
    """
    new_order = []

    # Remove the first stop from the unordered route
    remaining_route = unordered_route.drop(unordered_route[unordered_route.name == first_stop.name.values[0]].index).reset_index(drop=True)

    # Build an STRtree index for the remaining route
    tree = shapely.strtree.STRtree(remaining_route.geometry)

    # Append the first stop to the new order
    new_order.append(first_stop)

    for iter_ in range(len(remaining_route)):
        try:
            if iter_ == 0:
                # Find the nearest station to the first stop
                nearest_station = pd.DataFrame(remaining_route.iloc[tree.nearest(first_stop.geometry)[0]]).T
                new_order.append(nearest_station)
                remaining_route = remaining_route.drop(remaining_route[remaining_route.name == nearest_station.name.values[0]].index).reset_index(drop=True)
                tree = shapely.strtree.STRtree(remaining_route.geometry)
            elif iter_ == 1:
                # Find the second nearest station to the first stop
                second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(nearest_station.geometry)[0]]).T
                new_order.append(second_station)
                remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
                tree = shapely.strtree.STRtree(remaining_route.geometry)
            else:
                # Find the next nearest station based on the previous station
                second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(second_station.geometry)[0]]).T
                new_order.append(second_station)
                remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
                tree = shapely.strtree.STRtree(remaining_route.geometry)
        except TypeError:
            pass  # Ignore 'NoneType' object is not subscriptable error

    # Concatenate the ordered stops into a DataFrame
    ordered_route = pd.concat(new_order).reset_index(drop=True)
    # # Reverse the sequence to 
    # ordered_route = ordered_route.iloc[::-1].reset_index(drop=True)
    return ordered_route


# Orders stations based on the tram line routes and start station names
def order_stations_inline(tram_line_dict, all_tram_stations_name, tram_routes, tram_start_station_name_dict):
    """
    Orders the tram stations based on the tram line routes and start station names.

    Args:
        tram_line_dict (dict): A dictionary containing tram line information.
        all_tram_stations_name (DataFrame): A DataFrame containing all tram station names.
        tram_routes (DataFrame): A DataFrame containing tram route information.
        tram_start_station_name_dict (dict): A dictionary mapping tram line names to their respective start station names.

    Returns:
        dict: A dictionary containing ordered tram station routes for each tram line.
    """
    
    # Create a copy of the tram_line_dict dictionary
    tram_stations_dict = tram_line_dict.copy()

    # Iterate over each key-value pair in tram_stations_dict
    for key, value in tram_stations_dict.items():
        value1 = value
        # Filter the all_tram_stations_name DataFrame to include only stations within the buffer of the corresponding tram route
        value2 = all_tram_stations_name.loc[all_tram_stations_name.within(tram_routes.iloc[value1].geometry.buffer(0.000001))]
        # Assign new IDs to the filtered stations
        value2['id'] = value2.reset_index().index
        # Update the value in tram_stations_dict to include only necessary columns and reset the index
        tram_stations_dict[key] = value2[['id', 'name', 'geometry', 'geo_x', 'geo_y']].reset_index(drop=True)

    # Create copies of tram_line_dict for storing tram stations in order and ordered routes
    tram_stations_inorder_dict = tram_line_dict.copy()
    tram_order_route_dict = tram_line_dict.copy()

    # Iterate over each line in tram_stations_inorder_dict
    for line in tram_stations_inorder_dict.keys():
        tram_stations_inorder = tram_stations_dict[line]
        # Find the start station for the current line and update tram_stations_inorder_dict with only that station
        for i in range(len(tram_stations_inorder)):
            if tram_stations_inorder.iloc[i]['name'] == tram_start_station_name_dict[line]:
                tram_stations_inorder_dict[line] = pd.DataFrame(tram_stations_inorder.iloc[i]).T
        # Call the order_route function to order the remaining stations and store the result in tram_order_route_dict
        tram_order_route_dict[line] = order_route(tram_stations_inorder_dict[line], tram_stations_dict[line])

    return tram_order_route_dict,tram_stations_dict

In [13]:
# Create a copy of the tram_line_dict dictionary
city_sub_stations_dict = city_sub_line_dict.copy()

# Iterate over each key-value pair in tram_stations_dict
for key, value in city_sub_stations_dict.items():
    value1 = value
    # Filter the all_tram_stations_name DataFrame to include only stations within the buffer of the corresponding tram route
    value2 = city_all_sub_stations_name.loc[city_all_sub_stations_name.within(sub_routes.iloc[value1].geometry.buffer(0.000001))]
    # Assign new IDs to the filtered stations
    value2['id'] = value2.reset_index().index
    # Update the value in tram_stations_dict to include only necessary columns and reset the index
    city_sub_stations_dict[key] = value2[['id', 'name', 'geometry', 'geo_x', 'geo_y']].reset_index(drop=True)

# Create copies of tram_line_dict for storing tram stations in order and ordered routes
city_sub_stations_inorder_dict = city_sub_line_dict.copy()
city_sub_order_route_dict = city_sub_line_dict.copy()

city_sub_stations_inorder = city_sub_stations_dict['Metro 51: Centraal Station => Isolatorweg']
    # Find the start station for the current line and update tram_stations_inorder_dict with only that station
for i in range(len(city_sub_stations_inorder)):
    if city_sub_stations_inorder.iloc[i]['name'] == city_sub_start_station_name_dict['Metro 51: Centraal Station => Isolatorweg']:
        city_sub_stations_inorder_dict['Metro 51: Centraal Station => Isolatorweg'] = pd.DataFrame(city_sub_stations_inorder.iloc[i]).T
    # Call the order_route function to order the remaining stations and store the result in tram_order_route_dict
city_sub_order_route_dict['Metro 51: Centraal Station => Isolatorweg'] = order_route(city_sub_stations_inorder_dict['Metro 51: Centraal Station => Isolatorweg'], city_sub_stations_dict['Metro 51: Centraal Station => Isolatorweg'])

In [14]:
city_sub_start_station_name_dict['Metro 51: Centraal Station => Isolatorweg']

'Isolatorweg'

In [15]:
city_sub_stations_inorder_dict

{'Metro 50: Gein => Isolatorweg': 0,
 'Metro 50: Isolatorweg => Gein': 1,
 'Metro 51: Centraal Station => Isolatorweg':     id         name                      geometry     geo_x      geo_y
 20  20  Isolatorweg  POINT (4.8505958 52.3951031)  4.850596  52.395103,
 'Metro 51: Isolatorweg => Centraal Station': 3,
 'Metro 52: Noord => Zuid': 4,
 'Metro 52: Zuid => Noord': 5,
 'Metro 53: Centraal Station => Gaasperplas': 6,
 'Metro 53: Gaasperplas => Centraal Station': 7,
 'Metro 54: Centraal Station => Gein': 8,
 'Metro 54: Gein => Centraal Station': 9}

In [16]:
new_order = []

# Remove the first stop from the unordered route
remaining_route = city_sub_stations_dict['Metro 51: Centraal Station => Isolatorweg'].drop(city_sub_stations_dict['Metro 51: Centraal Station => Isolatorweg'][city_sub_stations_dict['Metro 51: Centraal Station => Isolatorweg'].name == city_sub_stations_inorder_dict['Metro 51: Centraal Station => Isolatorweg'].name.values[0]].index).reset_index(drop=True)

# Build an STRtree index for the remaining route
tree = shapely.strtree.STRtree(remaining_route.geometry)

# Append the first stop to the new order
new_order.append(city_sub_stations_inorder_dict['Metro 51: Centraal Station => Isolatorweg'])

for iter_ in range(len(remaining_route)):
    try:
        if iter_ == 0:
            # Find the nearest station to the first stop
            nearest_station = pd.DataFrame(remaining_route.iloc[tree.nearest(city_sub_stations_inorder_dict['Metro 51: Centraal Station => Isolatorweg'].geometry)[0]]).T
            new_order.append(nearest_station)
            remaining_route = remaining_route.drop(remaining_route[remaining_route.name == nearest_station.name.values[0]].index).reset_index(drop=True)
            tree = shapely.strtree.STRtree(remaining_route.geometry)
        elif iter_ == 1:
            # Find the second nearest station to the first stop
            second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(nearest_station.geometry)[0]]).T
            new_order.append(second_station)
            remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
            tree = shapely.strtree.STRtree(remaining_route.geometry)
        else:
            # Find the next nearest station based on the previous station
            second_station = pd.DataFrame(remaining_route.iloc[tree.nearest(second_station.geometry)[0]]).T
            new_order.append(second_station)
            remaining_route = remaining_route.drop(remaining_route[remaining_route.name == second_station.name.values[0]].index).reset_index(drop=True)
            tree = shapely.strtree.STRtree(remaining_route.geometry)
    except TypeError:
        pass  # Ignore 'NoneType' object is not subscriptable error

# Concatenate the ordered stops into a DataFrame
ordered_route = pd.concat(new_order).reset_index(drop=True)

In [17]:
ordered_route

,id,name,geometry,geo_x,geo_y
0,20,Isolatorweg,POINT (4.8505958 52.3951031),4.850596,52.395103
1,0,Station Sloterdijk,POINT (4.8389786 52.3890375),4.838979,52.389038
2,12,De Vlugtlaan,POINT (4.838233 52.3794484),4.838233,52.379448
3,13,Jan van Galenstraat,POINT (4.8353639 52.3725816),4.835364,52.372582
4,14,Postjesweg,POINT (4.8340343 52.3645962),4.834034,52.364596
5,11,Lelylaan,POINT (4.8344667 52.3580615),4.834467,52.358061
6,2,Heemstedestraat,POINT (4.834401 52.3523291),4.834401,52.352329
7,10,Henk Sneevlietweg,POINT (4.8345741 52.3463868),4.834574,52.346387
8,7,Amstelveenseweg,POINT (4.8575709 52.3384751),4.857571,52.338475
9,9,Zuid,POINT (4.8739633 52.3393223),4.873963,52.339322


In [18]:
city_sub_order_route_dict

{'Metro 50: Gein => Isolatorweg': 0,
 'Metro 50: Isolatorweg => Gein': 1,
 'Metro 51: Centraal Station => Isolatorweg':     id                 name                      geometry     geo_x      geo_y
 0   20          Isolatorweg  POINT (4.8505958 52.3951031)  4.850596  52.395103
 1    0   Station Sloterdijk  POINT (4.8389786 52.3890375)  4.838979  52.389038
 2   12         De Vlugtlaan   POINT (4.838233 52.3794484)  4.838233  52.379448
 3   13  Jan van Galenstraat  POINT (4.8353639 52.3725816)  4.835364  52.372582
 4   14           Postjesweg  POINT (4.8340343 52.3645962)  4.834034  52.364596
 5   11             Lelylaan  POINT (4.8344667 52.3580615)  4.834467  52.358061
 6    2      Heemstedestraat   POINT (4.834401 52.3523291)  4.834401  52.352329
 7   10    Henk Sneevlietweg  POINT (4.8345741 52.3463868)  4.834574  52.346387
 8    7      Amstelveenseweg  POINT (4.8575709 52.3384751)  4.857571  52.338475
 9    9                 Zuid  POINT (4.8739633 52.3393223)  4.873963  52.339322
 

In [19]:
city_sub_order_route_dict,tram_stations_dict = order_stations_inline(city_sub_line_dict, city_all_sub_stations_name, sub_routes, city_sub_start_station_name_dict)
city_sub_order_route_dict

{'Metro 50: Gein => Isolatorweg':     id                 name                      geometry     geo_x      geo_y
 0   21          Isolatorweg  POINT (4.8505958 52.3951031)  4.850596  52.395103
 1    0   Station Sloterdijk  POINT (4.8389786 52.3890375)  4.838979  52.389038
 2   13         De Vlugtlaan   POINT (4.838233 52.3794484)  4.838233  52.379448
 3   14  Jan van Galenstraat  POINT (4.8353639 52.3725816)  4.835364  52.372582
 4   15           Postjesweg  POINT (4.8340343 52.3645962)  4.834034  52.364596
 5   12             Lelylaan  POINT (4.8344667 52.3580615)  4.834467  52.358061
 6    2      Heemstedestraat   POINT (4.834401 52.3523291)  4.834401  52.352329
 7   11    Henk Sneevlietweg  POINT (4.8345741 52.3463868)  4.834574  52.346387
 8    8      Amstelveenseweg  POINT (4.8575709 52.3384751)  4.857571  52.338475
 9   10                 Zuid  POINT (4.8739633 52.3393223)  4.873963  52.339322
 10  19          Station RAI    POINT (4.889539 52.337689)  4.889539  52.337689
 11   9